<a href="https://colab.research.google.com/github/siddhant76/Demo/blob/master/first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import col, lit, when
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:
spark

In [41]:
data = [('James',None,'Smith','1991-04-01','M',3000),    #None is null
  ('Michael','Rose',None,'2000-05-19','M',4000),
  ('Robert',None,'Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
#df.printSchema()
df.show(truncate=True)

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|      null|   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|    null|2000-05-19|     M|  4000|
|   Robert|      null|Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [36]:
df1=df.filter(df.middlename.isNotNull())
df1.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|  Michael|      Rose|    null|2000-05-19|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [70]:
from pyspark.sql.functions import col, lit, when

In [74]:
#df1 = df.withColumn("salary",col("salary") *-8).select("salary")
df1 = df.withColumn("salary",when(df.salary <= 0, df.salary* -100).otherwise(df.salary)).select("salary")
#df1 = df.select("salary")

In [75]:
df1.show()
median = np.median(df1.collect())
median

+------+
|salary|
+------+
|  3000|
|  4000|
|  4000|
|  4000|
|   100|
+------+



4000.0

In [55]:
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|      null|   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|    null|2000-05-19|     M|  4000|
|   Robert|      null|Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [78]:
df_select = df.filter(df.middlename.isNotNull()).select('firstname','middlename')

In [79]:
df_select.show()

+---------+----------+
|firstname|middlename|
+---------+----------+
|  Michael|      Rose|
|    Maria|      Anne|
|      Jen|      Mary|
+---------+----------+

